# QCUT10 : ESelect quality cuts

- author Sylvie Dagoret-Campagne
- creation date 2026-02-04 : 
- last update : 2026-02-05
- Kernel @usdf **w_2026_02*
- Home emac : base (conda)
- laptop : conda_py313

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
# must install the mysitcom package by doing at top level "pip install --user -e . "
from mysitcom.auxtel.qualitycuts import scatter_datetime
from mysitcom.auxtel.qualitycuts import strip_datetime
from mysitcom.auxtel.qualitycuts import bar_counts_by_night
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_vs_time
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_vs_time_by_filter
from mysitcom.auxtel.qualitycuts import stripplot_target_vs_time
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_vs_time_by_target_filter
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_histo_by_target_filter
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_vs_time_by_target_filter_colorsedtype
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_histo_by_target_filter_colorsedtype
from mysitcom.auxtel.qualitycuts import summarize_dccd_chi2
from mysitcom.auxtel.qualitycuts import plot_param_histogram_grid
from mysitcom.auxtel.qualitycuts import plot_params_and_chi2_vs_time
from mysitcom.auxtel.qualitycuts import plot_param_chi2_correlation_grid
from mysitcom.auxtel.qualitycuts import plot_param2_vs_param1_colored_by_time
from mysitcom.auxtel.qualitycuts import plot_param_difference_vs_time
from mysitcom.auxtel.qualitycuts import plot_param_difference_vs_time_colored_by_chi2
from mysitcom.auxtel.qualitycuts import plot_single_param_vs_time_colored_by_chi2
from mysitcom.auxtel.qualitycuts import plot_single_param_vs_time
from mysitcom.auxtel.qualitycuts import plot_param_scatterandhistogram_grid
from mysitcom.auxtel.qualitycuts import plot_param_histogram_bytarget_grid
from mysitcom.auxtel.qualitycuts import save_param_histogram_bytarget_pdf
from mysitcom.auxtel.qualitycuts import plot_param_scatterandhistogram_pdf
from mysitcom.auxtel.qualitycuts import ParameterCutSelection,ParameterCutTools

In [ ]:
from mysitcom.auxtel.pwv import GetNightMidnightsDict
from mysitcom.auxtel.pwv import GetNightBoundariesDict
from mysitcom.auxtel.pwv import normalize_column_data_bytarget_byfilter
from mysitcom.auxtel.pwv import shiftaverage_column_data_byfilter
from mysitcom.auxtel.pwv import pwv_deviation_from_linear_interp_datetime
from mysitcom.auxtel.pwv import plot_atmparam_hist_per_filter

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figs_QCUT10"
prefix = "qcut10"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype

import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "xx-large"

import scipy
from scipy.optimize import curve_fit,least_squares

from pprint import pprint

# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time
from collections import OrderedDict

In [ ]:
from IPython.display import display, JSON
import json

In [ ]:
# Remove to run faster the notebook
#import ipywidgets as widgets
#%matplotlib widget

In [ ]:
from QCUT00_parameters import *

In [ ]:
DumpConfig()

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

### Configuration

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

In [ ]:
PWVMIN = 0.
PWVMAX = 20.

In [ ]:
FLAG_WITHCOLLIMATOR = False
DATE_WITHCOLLIMATOR = 20230930
datetime_WITHCOLLIMATOR = convertNumToDatestr(DATE_WITHCOLLIMATOR)
datetime_WITHCOLLIMATOR = pd.to_datetime("2023-09-30 00:00:00.0+0000")
datetime_WITHCOLLIMATOR

## Initialisation

### Read the file
- `atmfilename` is defined in `QCUT00_parameters.py` 

In [ ]:
the_suptitle = butlerusercollectiondict[version_run] 

In [ ]:
inputfilename = atmfilename.split("/")[-1]

if "parquet" in inputfilename:
    df_spec = pd.read_parquet(atmfilename)
elif "npy" in inputfilename:
    specdata = np.load(atmfilename,allow_pickle=True)
    specdata = np.load(atmfilename,allow_pickle=True)
    df_spec = pd.DataFrame(specdata)
    df_spec["D_CCD [mm]"] = df_spec["D2CCD"]
    df_spec["PWV [mm]"] = df_spec["PWV [mm]_x"] 
    df_spec["PWV [mm]_rum"] = df_spec["PWV [mm]_y"] 
    df_spec["PWV [mm]_err"] = df_spec["PWV [mm]_err_x"] 
    df_spec["PWV [mm]_err_rum"] = df_spec["PWV [mm]_err_y"] 


    cols = [
    "PWV [mm]",
    "PWV [mm]_rum",
    "PWV [mm]_err",
    "PWV [mm]_err_rum",
    ]

    df_spec = df_spec.dropna(subset=cols)
else:
    raise "bad path of filename {inputfilename}"
    

In [ ]:
the_suptitle = butlerusercollectiondict[version_run] 

In [ ]:
FLAG_RENAME_SPECTROGRAM_VARIABLES = True

if FLAG_RENAME_SPECTROGRAM_VARIABLES:
    df_spec.rename(
    {
    "chi2":"chi2_ram",
    "A1":"A1_ram",
    "A1_err": "A1_err_ram",
    "A2": "A2_ram",
    "A2_err": "A2_err_ram",
    "A3": "A3_ram",
    "A3_err": "A3_err_ram", 
    "VAOD": "VAOD_ram", 
    "VAOD_err": "VAOD_err_ram", 
    "angstrom_exp" : "angstrom_exp_ram", 
    "angstrom_exp_err" : "angstrom_exp_err_ram" , 
    "ozone [db]" :"ozone [db]_ram", 
    "ozone [db]_err": "ozone [db]_err_ram", 
    "PWV [mm]":  "PWV [mm]_ram",
    "PWV [mm]_err":"PWV [mm]_err_ram" , 
    "B": "B_ram" , 
    "B_err" : "B_err_ram", 
    "A_star": "A_star_ram" , 
    "A_star_err": "A_star_err_ram" , 
    "D_CCD [mm]" : "D_CCD [mm]_ram", 
    "D_CCD [mm]_err": "D_CCD [mm]_err_ram" 
    }
    ,axis=1,inplace = True)

In [ ]:
df_spec

In [ ]:
print(" , ".join(df_spec.columns)) 

In [ ]:
#df_spec.dtypes.to_frame('Type de donnée')

In [ ]:
# add time for plotting
#df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"],utc=True)

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
df_spec["seq_num"]  = df_spec["id"] % 100_000

In [ ]:
df_spec[["id","FILTER"]]

### Convert DATE-OBS to pd_to_datetime

In [ ]:
df_spec["DATE-OBS"] = pd.to_datetime(
    df_spec["DATE-OBS"],
    utc=True,
    errors="coerce").dt.tz_convert(None)

In [ ]:
df_spec["FILTER"].unique()

In [ ]:
print(list(df_spec.columns))

## Targets list and target color palette

In [ ]:
# Comptage et tri
target_counts = (
    df_spec['TARGET']
    .value_counts()
    .sort_values(ascending=False)
)
targets = target_counts.index.tolist()
counts = target_counts.values

In [ ]:
chosen_palette = "tab20"

if chosen_palette == "husl":
    palette = sns.color_palette("husl", n_colors=len(targets))
    #palette = sns.color_palette("husl", len(targets))[::-1]
    target_color_map = OrderedDict(zip(targets, palette))
elif chosen_palette == "hsv":
    base_palette = sns.color_palette("hsv", n_colors=len(targets))
    # réordonnancement pour maximiser contraste local
    order = np.arange(len(base_palette))
    order = np.roll(order, len(order)//2)
    palette = [base_palette[i] for i in order]
    target_color_map = OrderedDict(zip(targets, palette))
    #target_color_map = OrderedDict(zip(targets, palette[::-1]))
elif chosen_palette == "tab20":
    palette = sns.color_palette("tab20b", 20) + sns.color_palette("tab20c", 10)
    palette = palette[:len(targets)]
    target_color_map = OrderedDict(zip(targets, palette))
    #target_color_map = OrderedDict(zip(targets, palette[::-1]))
else:
    palette = sns.color_palette("viridis", n_colors=len(targets))
    #palette = sns.color_palette("viridis", n_colors=len(targets))[::-1]
    target_color_map = OrderedDict(zip(targets, palette)) 

# Colormap discrete
cmap = mpl.colors.ListedColormap(palette)
norm = mpl.colors.BoundaryNorm(boundaries=range(len(targets)+1),ncolors=len(targets))

In [ ]:
ordered_list_of_targets = list(target_color_map.keys())

In [ ]:
# palette
#target_color_map = OrderedDict( zip(targets, palette))

In [ ]:
fig = plt.figure(figsize=(0.6*len(targets), 3),layout="constrained")

# axe très épais (0.15)
cax = fig.add_axes([0.05, 0.15, 0.9, 0.15])  
# [left, bottom, width, height]

cb = mpl.colorbar.ColorbarBase(
    cax,
    cmap=cmap,
    norm=norm,
    orientation='horizontal'
)

cb.set_ticks([i + 0.5 for i in range(len(targets))])
cb.set_ticklabels(targets)
cb.ax.tick_params(labelrotation=90)
cb.set_label("TARGET", labelpad=10)

cb.ax.tick_params(labelsize=20,length=6,width=1.5)

figfilename = f"{pathfigs}/{prefix}_palette_{chosen_palette}_targetnames{figtype}"
fig.savefig(figfilename)

fig.show()


In [ ]:
fig, ax = plt.subplots(figsize=(6, 0.3*len(targets)))

sns.barplot(
    x=counts,
    y=targets,
    palette=palette,
    ax=ax
)

ax.set_xlabel("Number of Obs")
ax.set_ylabel("TARGET")
ax.set_title("TARGET observed")

plt.tight_layout()

figfilename = f"{pathfigs}/{prefix}_baplottargets_palette_{chosen_palette}{figtype}"
plt.savefig(figfilename)

plt.show()

## Processing before cut studies

### PWV difference and PWV relative ratio

In [ ]:
denom = np.sqrt(df_spec["PWV [mm]_err_ram"]**2 + df_spec["PWV [mm]_err_rum"]**2)

df_spec["diff_PWV_norm"] = np.where(
    np.isfinite(denom) & (denom > 0),
    (df_spec["PWV [mm]_ram"] - df_spec["PWV [mm]_rum"]) / denom,
    np.nan
)

df_spec["diff_PWV"] =  (df_spec["PWV [mm]_ram"] - df_spec["PWV [mm]_rum"]) 
df_spec["diff_PWV_err"] = np.sqrt( (df_spec["PWV [mm]_err_ram"]**2 - df_spec["PWV [mm]_err_rum"]**2)) 

### Normalised chi2

In [ ]:
df_spec, df1 = normalize_column_data_bytarget_byfilter(df_spec,target_col="TARGET",filter_col="FILTER",feature_col= "CHI2_FIT",ext="norm")
df_spec, df2 = normalize_column_data_bytarget_byfilter(df_spec,target_col="TARGET",filter_col="FILTER",feature_col= "chi2_ram",ext="norm")
df_spec, df3 = normalize_column_data_bytarget_byfilter(df_spec,target_col="TARGET",filter_col="FILTER",feature_col= "chi2_rum",ext="norm")

### Angle uniformization

In [ ]:
#df["angle_180"] = ((df["angle_360"] + 180) % 360) - 180

In [ ]:
flag_angles_m180_p180 = True

In [ ]:
if flag_angles_m180_p180:
    df_spec["DOMEAZ"] = ((df_spec["DOMEAZ"] + 180) % 360) - 180
    df_spec["RA"] = ((df_spec["RA"] + 180) % 360) - 180
    df_spec["WINDDIR"] = ((df_spec["WINDDIR"] + 180) % 360) - 180
    flag_angles_m180_p180 = True

In [ ]:
df_spec["WINDSPDPARR"] =  df_spec["WINDSPD"]*np.cos(df_spec["AZ"]-df_spec["WINDDIR"])
df_spec["WINDSPDPERP"] =  df_spec["WINDSPD"]*np.sin(df_spec["AZ"]-df_spec["WINDDIR"])

## What to keep

In [ ]:
columns_keep = ["id","Time","TARGET","ROTANGLE","D2CCD", "DOMEAZ","AZ","EL","WINDSPD", "WINDDIR","PARANGLE","TARGETX","TARGETY","CHI2_FIT_norm","PIXSHIFT","PSF_REG","TRACE_R", 
"A2_FIT", "AM_FIT", "MEANFWHM", "AIRMASS", "OUTTEMP", "OUTPRESS", "OUTHUM","FILTER", "CAM_ROT","chi2_ram_norm","A1_ram", "A2_ram", "A3_ram", "PWV [mm]_ram" ,"PWV [mm]_err_ram","B_ram",
"A_star_ram","D_CCD [mm]_ram","shift_x [pix]","shift_y [pix]", "angle [deg]", "P [hPa]","gamma_0_1", "gamma_1_1","gamma_2_1", "alpha_0_1","alpha_1_1","saturation_0_1",
"gamma_0_2","gamma_1_2","gamma_2_2", "alpha_0_2", "alpha_1_2", "alpha_2_2", "saturation_0_2", "chi2_rum_norm", "A1_rum", "A2_rum",
"PWV [mm]_rum","PWV [mm]_err_rum" ,"reso [nm]", "D_CCD [mm]_rum", "alpha_pix [pix]", "mount_motion_image_degradation_x",
"mount_motion_image_degradation_az_x", "mount_motion_image_degradation_el_x", "mount_jitter_rms_x","mount_jitter_rms_az_x", "mount_jitter_rms_el_x", "mount_jitter_rms_rot_x",
"dimm_seeing_x", "focus_z_x" ,"mount_motion_image_degradation_y", "mount_motion_image_degradation_az_y","diff_PWV","diff_PWV_err","abs_delta_PWV","PWV [mm]_shift","PWV [mm]_rum_shift"]

## Histograms of parameters

In [ ]:
params = [ 
    "alpha_0_1", 
    "alpha_1_1", 
    "alpha_0_2", 
#    "alpha_1_2", 
#    "alpha_2_2",
    "gamma_0_1",
    "gamma_1_1",
    "gamma_2_1",
    "angle [deg]", 
    "alpha_pix [pix]",
    "reso [nm]",
    #"shift_x[pix]",
    #"shift_y[pix]",
    'MEANFWHM',
    'PIXSHIFT',
    'PSF_REG',
    'TRACE_R',
    'CHI2_FIT_norm', 
    'chi2_ram_norm',
    'chi2_rum_norm',
    'D2CCD',
    'D_CCD [mm]_ram',
    'D_CCD [mm]_rum',
    'alpha_pix [pix]',
    "WINDSPD",
    "WINDDIR",
    "WINDSPDPARR",
    "WINDSPDPERP",
    "CAM_ROT",
    "ROTANGLE",
    "PARANGLE",
    "DOMEAZ",
    "AZ",
    "EL",
    "PARANGLE",
    "AIRMASS", 
    "OUTTEMP", 
    "OUTPRESS",
    "P [hPa]"
]

In [ ]:
param_ranges = {
    "alpha_0_1": (0, 5),
    "alpha_1_1": (-1, 1),
    "alpha_0_2": (0, 5),
    "gamma_0_1": (-2, 10),
    "gamma_1_1": (-5, 5),
    "gamma_2_1": (-2, 5),
    "angle [deg]": (0.1, 0.4),
    "reso [nm]": (0, 5),
    "MEANFWHM": (0, 30),
    "PIXSHIFT": (-1, 1),
    "PSF_REG": (0, 10),
    "TRACE_R": (0, 80),
    "CHI2_FIT_norm": (0, 3),
    "chi2_ram_norm": (0, 3),
    "chi2_rum_norm": (0, 3),
    "D2CCD": (186, 189),
    "D_CCD [mm]_ram": (186, 189),
    "D_CCD [mm]_rum": (186, 189),
    "ROTANGLE": (0.1, 0.3),
    "P [hPa]": (0, 2000),
}

filter_order = ["empty", "BG40_65mm_1", "OG550_65mm_1"]

In [ ]:
cuts = ParameterCutTools.generate_default_paramcuts()

In [ ]:
#cuts = {
#    param: {
#        filt: {"min": vmin, "max": vmax}
#        for filt in filter_order
#    }
#    for param, (vmin, vmax) in param_ranges.items()
#}

In [ ]:
#print(json.dumps(cuts, indent=4, sort_keys=True))

In [ ]:
display(JSON(cuts))

In [ ]:
filename_cuts_defaults = "cuts_default.json" 
ParameterCutTools.write_cuts_json(cuts,filename_cuts_defaults)

In [ ]:
saved_cuts = ParameterCutTools.load_cuts_json(filename_cuts_defaults)

In [ ]:
#print(json.dumps(saved_cuts, indent=4, sort_keys=False))

In [ ]:
selector = ParameterCutSelection(
    df_spec,
    params= params,
    id_col="id"
)

flags = selector.apply_cuts(cuts)
stats = selector.selection_statistics(cuts)

df_selected = df_spec.merge(flags, on="id")
df_keep = df_selected[df_selected["pass_all_cuts"]]

In [ ]:
flags.head()

In [ ]:
stats

In [ ]:
target_order = (
        stats
        .groupby("TARGET")["n_total"]
        .sum()
        .sort_values(ascending=False)
        .index
        )

In [ ]:
target_order

In [ ]:
def plot_selection_fraction_by_filter(
    stat,
    target_color_map,
    filter_order=None,
    figsize_per_filter=(6, 0.35),
):
    """
    Horizontal bar plot of selection fraction per TARGET, grouped by FILTER.
    """

    if filter_order is None:
        filter_order = stat["FILTER"].unique()

    n_filters = len(filter_order)

    # figure height adapts to number of targets
    n_targets = stat["TARGET"].nunique()
    fig_height = max(4, figsize_per_filter[1] * n_targets)

    fig, axes = plt.subplots(
        1,
        n_filters,
        figsize=(figsize_per_filter[0] * n_filters, fig_height),
        sharey=True,
    )

    if n_filters == 1:
        axes = [axes]

    target_order = (
        stat
        .groupby("TARGET")["n_total"]
        .sum()
        .sort_values(ascending=False)
        .index
        )


    for ax, filt in zip(axes, filter_order):
        df_f = stat[stat["FILTER"] == filt].copy()

        # impose the same TARGET order for all filters
        df_f = (
            df_f
            .set_index("TARGET")
            .reindex(target_order)
            .reset_index()
            )


        # sort targets for readability
        #df_f = df_f.sort_values("frac_pass_all")

        y = np.arange(len(df_f))

        colors = [
            target_color_map.get(t, "gray")
            for t in df_f["TARGET"]
        ]

        ax.barh(
            y,
            df_f["frac_pass_all"],
            color=colors,
            edgecolor="black",
            alpha=0.9,
        )

        ax.set_title(filt)
        ax.set_xlim(0, 1.0)
        ax.grid(axis="x", alpha=0.3)

        ax.set_yticks(y)
        ax.set_yticklabels(df_f["TARGET"])

        ax.set_xlabel("Selected fraction")
        ax.invert_yaxis()

    axes[0].set_ylabel("TARGET")

    plt.tight_layout()
    return fig


In [ ]:
fig = plot_selection_fraction_by_filter(
    stats,
    target_color_map,
    filter_order=["empty", "BG40_65mm_1", "OG550_65mm_1"],
)

plt.show()


In [ ]:
def plot_target_param_cuts(
    df,
    target,
    cuts,
    filter_value=None,
    target_color=None,
):
    """
    Barplot montrant la fraction de sélection pour chaque paramètre
    pour une TARGET donnée.
    
    df : dataframe original
    target : str, nom du target
    cuts : dict, structure cuts[param][filter] = {'min':..,'max':..}
    filter_value : str ou None, si on veut filtrer un filtre spécifique
    target_color : couleur pour la barre
    """

    df_t = df[df["TARGET"] == target].copy()
    
    if filter_value is not None:
        df_t = df_t[df_t["FILTER"] == filter_value]
    
    params = [p for p in cuts.keys() if p in df_t.columns]
    
    results = []
    
    for p in params:
        # applique la coupure pour ce paramètre
        if filter_value is not None:
            minv = cuts[p][filter_value].get("min")
            maxv = cuts[p][filter_value].get("max")
        else:
            # appliquer un "or" sur tous les filtres ?
            # ici on prend le min/max de la première occurrence
            minv = list(cuts[p].values())[0].get("min")
            maxv = list(cuts[p].values())[0].get("max")
        
        mask = pd.Series(True, index=df_t.index)
        if minv is not None:
            mask &= df_t[p] >= minv
        if maxv is not None:
            mask &= df_t[p] <= maxv
        
        n_pass = mask.sum()
        n_total = len(df_t)
        frac_pass = n_pass / n_total if n_total > 0 else 0
        
        results.append((p, n_pass, n_total, frac_pass))
    
    df_res = pd.DataFrame(results, columns=["param","n_pass","n_total","frac_pass"])
    
    # plot
    fig, ax = plt.subplots(figsize=(max(6, len(params)*0.6),4))
    
    ax.bar(df_res["param"], df_res["frac_pass"], color=target_color or "steelblue", edgecolor="black")
    ax.set_ylim(0,1)
    ax.set_ylabel("Fraction sélectionnée")
    ax.set_xlabel("Paramètre")
    ax.set_title(f"Target: {target}" + (f" | Filter: {filter_value}" if filter_value else ""))
    ax.set_xticklabels(df_res["param"], rotation=45, ha="right")
    ax.grid(axis="y", alpha=0.3)
    
    plt.tight_layout()
    return fig, df_res


In [ ]:
fig, df_frac = plot_target_param_cuts(
    df_spec,
    target="HD185975",
    cuts=cuts,
    filter_value="empty",
    target_color=target_color_map["HD185975"]
)
plt.show()


In [ ]:
fig, df_frac = plot_target_param_cuts(
    df_spec,
    target="HD36780",
    cuts=cuts,
    filter_value="empty",
    target_color=target_color_map["HD36780"]
)
plt.show()


In [ ]:
fig, df_frac = plot_target_param_cuts(
    df_spec,
    target="HD36780",
    cuts=cuts,
    filter_value="OG550_65mm_1",
    target_color=target_color_map["HD36780"]
)
plt.show()


In [ ]:
def plot_target_param_cuts_multi_filters(
    df,
    target,
    cuts,
    filter_order=None,
    target_color="steelblue",
    figsize_per_subplot=(16,3)
):
    """
    Plot vertical barplots of fraction of selection per parameter
    for a given TARGET, one subplot per filter (aligned x-axis).
    
    df : dataframe original
    target : str
    cuts : dict, cuts[param][filter] = {'min':..,'max':..}
    filter_order : list of filters to show
    target_color : color of bars
    figsize_per_subplot : tuple(width,height) for each subplot
    """
    
    # dataframe du target
    df_t = df[df["TARGET"] == target].copy()
    
    # si filter_order non fourni, on prend tous les filtres présents
    if filter_order is None:
        filter_order = df_t["FILTER"].unique()
    
    n_filters = len(filter_order)
    params = [p for p in cuts.keys() if p in df_t.columns]
    
    fig, axes = plt.subplots(
        n_filters,
        1,
        figsize=(figsize_per_subplot[0], figsize_per_subplot[1]*n_filters),
        sharex=True
    )
    
    if n_filters == 1:
        axes = [axes]
    
    for ax, filt in zip(axes, filter_order):
        df_f = df_t[df_t["FILTER"] == filt].copy()
        
        results = []
        for p in params:
            if filt in cuts[p]:
                minv = cuts[p][filt].get("min")
                maxv = cuts[p][filt].get("max")
            else:
                minv = None
                maxv = None
            
            mask = pd.Series(True, index=df_f.index)
            if minv is not None:
                mask &= df_f[p] >= minv
            if maxv is not None:
                mask &= df_f[p] <= maxv
            
            n_pass = mask.sum()
            n_total = len(df_f)
            frac_pass = n_pass / n_total if n_total > 0 else 0
            results.append(frac_pass)
        
        ax.bar(params, results, color=target_color, edgecolor="black")
        ax.set_ylim(0,1)
        ax.set_ylabel(f"{filt}")
        ax.grid(axis="y", alpha=0.3)
    
    axes[-1].set_xticklabels(params, rotation=45, ha="right")
    axes[-1].set_xlabel("Parameter")
    
    fig.suptitle(f"Target: {target} : fraction of selected events", fontsize=14)
    plt.tight_layout(rect=[0,0,1,0.96])
    
    return fig


In [ ]:
fig = plot_target_param_cuts_multi_filters(
    df_spec,
    target="HD36780",
    cuts=cuts,
    filter_order=["empty", "BG40_65mm_1", "OG550_65mm_1"],
    target_color=target_color_map["HD36780"]
)
plt.show()


In [ ]:
fig = plot_target_param_cuts_multi_filters(
    df_spec,
    target="HD185975",
    cuts=cuts,
    filter_order=["empty", "BG40_65mm_1", "OG550_65mm_1"],
    target_color=target_color_map["HD185975"]
)
plt.show()
